In [1]:
import requests
from bs4 import BeautifulSoup
import math
import pandas as pd
from urllib.parse import quote
import re
from datetime import datetime

In [5]:
base_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?'

In [3]:
with open('./../expo/proxies/working_proxies.txt','r') as f:
    proxylist = f.read().split('\n')

In [4]:
proxylist

['189.203.18.58:3128', '167.114.107.37:80', '', '']

In [ ]:
start = 0
keywords = input('Search for : ')
location = input('Location : ')
jobs_scraped = 0

target_url = base_url+f'keywords={quote(keywords)}&location={quote(location)}'

l=[]
o={}
k=[]

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

print('Job IDs scraped : ')

while start != -1:
    
    res = requests.get(target_url+f'&start={start}')

    if len(re.sub('[\n <!->]','',res.text))>0:

        soup=BeautifulSoup(res.text,'html.parser')
        alljobs_on_this_page=soup.find_all("li")
        
        for x in range(0,len(alljobs_on_this_page)):
            try : 
                jobid = alljobs_on_this_page[x].find("div",{"class":"base-card"}).get('data-entity-urn').split(":")[3]
                l.append(jobid)
            except :
                pass
    
        print(f'{len(l)}')
        start += 25
    
    else :
        start = -1

In [72]:
m = l

In [73]:
l = l[:20]

In [74]:
job_posting_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'

for j in range(0,len(l)):

    resp = requests.get(job_posting_url.format(l[j]))
    soup=BeautifulSoup(resp.text,'html.parser')

    try:
        o["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
    except:
        o["company"]=None

    try:
        o["location"]=soup.find("div",{"class":"topcard__flavor-row"}).find("span",{'class':'topcard__flavor--bullet'}).text.strip()
    except:
        o["location"]=None
    
    try:
        o['posted_time_ago']=soup.find_all("div",{"class":"topcard__flavor-row"})[1].find('span',{'class':'posted-time-ago__text'}).text.strip()
    except:
        o['posted_time_ago']=None
    
    try:
        o['applicant_number']=soup.find_all("div",{"class":"topcard__flavor-row"})[1].find('span',{'class':'num-applicants__caption'}).text.strip()
    except:
        o['applicant_number']=None

    try:
        o["job_title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
    except:
        o["job_title"]=None

    try :
        o["job_description"]=soup.find("div",{"class":"description__text description__text--rich"}).text.strip()
    except:
        o["job_description"]=None

    try:
        o["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
    except:
        o["level"]=None
    try:
        o["employment_type"]=soup.find("ul",{"class":"description__job-criteria-list"}).find_all("li")[1].text.replace("Employment type","").strip()
    except:
        o["employment_type"]=None
    try:
        o["job_function"]=soup.find("ul",{"class":"description__job-criteria-list"}).find_all("li")[2].text.replace("Job function","").strip()
    except:
        o["job_function"]=None
    try:
        o["industries"]=soup.find("ul",{"class":"description__job-criteria-list"}).find_all("li")[3].text.replace("Industries","").strip()
    except:
        o["industries"]=None

    k.append(o)
    o={}

df = pd.DataFrame(k)
timestamp = str(datetime.now())
df.to_csv(f'./../expo/{keywords}_{location}_{timestamp.split(".")[0]}_linkedinjobs.csv', index=False, encoding='utf-8')
print(len(df))

20


In [52]:
resp=requests.get('https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3821317085')

In [53]:
soup=BeautifulSoup(resp.text,'html.parser')
soup.find("div",{"class":"description__text description__text--rich"}).text.strip()

'We are on the lookout for a Data Analyst with a specialization in marketing analytics. In this critical role, you will harness the power of tools like Google Data Studio, BigQuery, and Power BI to derive actionable insights that drive our marketing decisions. Your expertise in analyzing complex datasets and turning them into understandable and strategic information will be essential in shaping our marketing strategies. This position is ideal for someone who thrives on data-driven challenges and has a passion for leveraging technology to inform business decisions.What You’ll Be DoingData Analysis and Interpretation: Utilize tools like Google Data Studio, BigQuery, and Power BI to analyze marketing data and extract meaningful insights. Developing Reporting Dashboards: Create dynamic dashboards and reports that provide real-time insights into marketing performance. Collaborating with Marketing Teams: Work closely with marketing teams to understand their data needs and provide insights th

In [86]:
str(timestamp).split(':')[0]+':'+str(timestamp).split(':')[1]

'2024-02-03 20:07'

In [6]:
df = pd.read_csv('/Users/tzikos/Desktop/python tasks/linkedin_job_scrape/expo/csv/chef_athens_2024-02-03 21:37_linkedinjobs.csv')

In [31]:
import requests
import random
import concurrent.futures

#opens a csv file of proxies and prints out the ones that work with the url in the extract function

with open('./../expo/proxies/proxy-list.txt', 'r') as f:
    proxylist = [line.strip() for line in f]

def extract(proxy):
    #this was for when we took a list into the function, without conc futures.
    #proxy = random.choice(proxylist)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0'}
    try:
        #change the url to https://httpbin.org/ip that doesnt block anything
        r = requests.get('https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3821317085', headers=headers, proxies={'http' : proxy,'https': proxy}, timeout=2)
    except:
        pass
    return proxy,r.status_code


In [27]:

with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(extract, proxylist)

In [45]:
import requests
from bs4 import BeautifulSoup
import random
import concurrent.futures

#get the list of free proxies
def getProxies():
    r = requests.get('https://free-proxy-list.net/')
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find('tbody')
    proxies = []
    for row in table:
        if row.find_all('td')[4].text =='elite proxy':
            proxy = ':'.join([row.find_all('td')[0].text, row.find_all('td')[1].text])
            proxies.append(proxy)
        else:
            pass
    return proxies

def extract(proxy):
    #this was for when we took a list into the function, without conc futures.
    #proxy = random.choice(proxylist)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0'}
    try:
        #change the url to https://httpbin.org/ip that doesnt block anything
        r = requests.get('https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3821317085', headers=headers, proxies={'http' : proxy,'https': proxy}, timeout=2).status_code
    except :
        r = 'failed'
    return proxy, r

proxylist = getProxies()
#print(len(proxylist))


In [46]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = [executor.submit(extract, proxy) for proxy in proxylist]

In [50]:
working_proxies = [result.result()[0] for result in results if result.result()[1]==200]

In [51]:
working_proxies

['117.250.3.58:8080', '189.203.18.58:3128']

In [ ]:
def extract(proxy):
    #this was for when we took a list into the function, without conc futures.
    #proxy = random.choice(proxylist)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0'}
    try:
        r = requests.get('https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?', headers=headers, proxies={'http' : proxy,'https': proxy}, timeout=1)
    except requests.ConnectionError as err:
        pass
    return proxy, r.status_code

proxylist = getProxies()
#print(len(proxylist))

#check them all with futures super quick
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = [executor.submit(extract, proxy) for proxy in proxylist]

working_proxies = []
for result in results:
    try:
        working_proxies.append(result.result())
    except:
        pass

with open('./expo/proxies/scraped_working_proxies.txt', 'w') as file:
    # Write each element to the file followed by a newline character
    for wp in working_proxies:
        file.write("%s\n" % wp)